Given a string s which represents an expression, evaluate this expression and return its value. 

The integer division should truncate toward zero.

You may assume that the given expression is always valid. All intermediate results will be in the range of [-231, 231 - 1].

Note: You are not allowed to use any built-in function which evaluates strings as mathematical expressions, such as eval().

 

Example 1:

Input: s = "3+2*2"
Output: 7
Example 2:

Input: s = " 3/2 "
Output: 1
Example 3:

Input: s = " 3+5 / 2 "
Output: 5
 

Constraints:

1 <= s.length <= 3 * 105
s consists of integers and operators ('+', '-', '*', '/') separated by some number of spaces.
s represents a valid expression.
All the integers in the expression are non-negative integers in the range [0, 231 - 1].
The answer is guaranteed to fit in a 32-bit integer.

# brute force 
- Iterate through the string character by character.
- Maintain a current number and the last operator seen.
- Use a stack to hold numbers:
    - When you see +, push the number.
    - When you see -, push -number.
    - When you see * or /, pop the top of the stack, apply the operation, push the result back.

In [ ]:
class Solution:
    def calculate(self, s: str) -> int:
        stack = []
        num = 0
        sign = '+'
        s = s.replace(' ', '')  # remove spaces
        
        for i, ch in enumerate(s):
            if ch.isdigit():
                # there can be numbers 123, so you have to build the number.
                num = num * 10 + int(ch)

            if not ch.isdigit() or i == len(s) - 1:
                if sign == '+':
                    stack.append(num)
                elif sign == '-':
                    stack.append(-num)
                elif sign == '*':
                    stack[-1] = stack[-1] * num
                elif sign == '/':
                    # Python division truncates toward 0 for negative numbers like in Leetcode
                    stack[-1] = int(stack[-1] / num)
                
                # this is to track what was hte previous sign.
                # 3*2/2 .. here when we are at 3ed index. the stack will be [3, 2] and sign will be *.
                # now we are at the '/' sign .. we have to calculate the previous sign. 3 * 2 and push the result.
                sign = ch
                num = 0

        return sum(stack)


# tc - O(n)
# sc - O(n) for stack

exp = "3+6/2*1-0"


| Step | Char | Current Num | Sign | Stack Before | Action                                                      | Stack After |
| ---- | ---- | ----------- | ---- | ------------ | ----------------------------------------------------------- | ----------- |
| 1    | '3'  | 3           | '+'  | \[]          | Building number                                             | \[]         |
| 2    | '+'  | 3           | '+'  | \[]          | Previous sign is '+', push `+3` to stack                    | \[3]        |
| 3    | '6'  | 6           | '+'  | \[3]         | Building number                                             | \[3]        |
| 4    | '/'  | 6           | '+'  | \[3]         | Previous sign is '+', push `+6` to stack                    | \[3, 6]     |
| 5    | '2'  | 2           | '/'  | \[3, 6]      | Building number                                             | \[3, 6]     |
| 6    | '\*' | 2           | '/'  | \[3, 6]      | Previous sign is '/', do `6 / 2 = 3`, replace top of stack  | \[3, 3]     |
| 7    | '1'  | 1           | '\*' | \[3, 3]      | Building number                                             | \[3, 3]     |
| 8    | '-'  | 1           | '\*' | \[3, 3]      | Previous sign is '\*', do `3 * 1 = 3`, replace top of stack | \[3, 3]     |
| 9    | '0'  | 0           | '-'  | \[3, 3]      | Building number                                             | \[3, 3]     |
| 10   | END  | 0           | '-'  | \[3, 3]      | Previous sign is '-', push `-0` to stack                    | \[3, 3, 0]  |


In [ ]:
# solution without using stack - optimal approach
# We apply * and / directly to last_num as soon as we see the operator.
# We only add last_num to the result when we see a + or -.


# page 36: in string algo note book.
class Solution:
    def calculate(self, s: str) -> int:
        s = s.replace(" ", "")
        n = len(s)
        current_num = 0
        last_num = 0
        result = 0
        operation = '+'

        for i in range(n):
            char = s[i]
            print(char)

            if char.isdigit():
                current_num = current_num * 10 + int(char)

            if not char.isdigit() or i == n - 1:
                if operation == '+':
                    result += last_num
                    last_num = current_num
                elif operation == '-':
                    result += last_num
                    last_num = -current_num
                elif operation == '*':
                    last_num = last_num * current_num
                elif operation == '/':
                    last_num = int(last_num / current_num)  # truncate toward zero

                operation = char
                current_num = 0

        result += last_num
        return result
    
# tc- O(n)
# sc - O(1) since we are not using any extra space except for a few



In [4]:
Solution().calculate("3+6/2*1-2")

3
+
6
/
2
*
1
-
2


4

exp = "3+6/2*1-0"



| Step | Char | Current Num | Operation | Last Num | Result | Action                             |
| ---- | ---- | ----------- | --------- | -------- | ------ | ---------------------------------- |
| 1    | 3    | 3           | +         | 0        | 0      | building number                    |
| 2    | +    | 3           | +         | 0        | 0      | apply `+` → last\_num = 0          |
| 3    | 6    | 6           | +         | 3        | 0      | building number                    |
| 4    | /    | 0           | /         | 6        | 3      | apply `+` → result += 3, reset     |
| 5    | 2    | 2           | /         | 6        | 3      | building number                    |
| 6    | \*   | 0           | \*        | 3        | 3      | apply `/` → last\_num = 6 // 2 = 3 |
| 7    | 1    | 1           | \*        | 3        | 3      | building number                    |
| 8    | -    | 0           | -         | 3        | 3      | apply `*` → last\_num = 3 \* 1 = 3 |
| 9    | 0    | 0           | -         | 3        | 6      | apply `+` → result += 3            |
| End  | end  | 0           | -         | 0        | 6      | apply `-` → result += (-0)         |
